In [1]:
import torch
import torch.nn as nn
import pickle
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from exp.exp_anomaly_detection_custom import Exp_Anomaly_Detection_custom

In [2]:
def load(settings):
    with open("./test_results/" + settings + "/args.pkl", "rb") as f:
        args = pickle.load(f)
    
    with open("./test_results/" + settings + "/threshold.txt") as f:
        threshold = float(f.read().strip())
    
    exp = Exp_Anomaly_Detection_custom(args)  
    model = exp._build_model()
    model.load_state_dict(torch.load("./checkpoints/" + settings + "/checkpoint.pth"))
    
    return args, model, threshold

In [3]:
def predict(args, df, target, model, threshold=None):
    """
    args       : argparse avec seq_len, device etc
    df         : DataFrame contenant au moins la colonne `target`
    target     : nom de la variable à analyser
    model      : modèle TimesNet déjà chargé
    threshold  : seuil pour flagger les anomalies
    """

    ###### Normalisation 
    scaler = StandardScaler()
    data = scaler.fit_transform(df[[target]].values)

    ###### Découpage en fenêtres
    seq_len = args.seq_len
    X = []
    i = 0

    while i <= len(data):
        if i + seq_len > len(data):
           X.append(data[len(data) - seq_len:len(data)]) 
        else:
            X.append(data[i:i+seq_len])
        i += seq_len  

    X = np.array(X)  # shape = (nb_seq, seq_len, 1)

    ###### Passage par batch dans le modèle 
    anomaly_criterion = nn.MSELoss(reduction="none")
    model = model.to(args.device)
    all_scores, all_recon = [], []

    with torch.no_grad():
        for i in range(0, len(X)):
            print(f"Processing batch {i+1}/{len(X)} with shape {X[i].shape}")
            batch = torch.tensor([X[i]], dtype=torch.float32) # (1, seq_len, 1)
            batch = batch.to(args.device)  
            output = model(batch, None, None, None)  # reconstruction

            # Erreur
            loss = torch.mean(anomaly_criterion(batch, output), dim=-1).cpu().numpy()
            if i + 1 == len(X):
                l = loss[0][seq_len - (len(data) % seq_len):].reshape(-1, 1)
                print(f'loss shape {l.shape}')
                all_scores.append(l)  # Last batch may not be full
            else:
                l = loss[0].reshape(-1, 1)
                print(f'loss shape {l.shape}')
                all_scores.append(l)  # Reshape to (seq_len, 1)

            # Reconstruction → back to numpy
            recon = output.cpu().numpy()
            if i + 1 == len(X):
                r = recon[0][seq_len - (len(data)  % seq_len):]
                print(f'reconstruction shape : {r.shape}')
                all_recon.append(r)
            else:
                r = recon[0]
                print(f'reconstruction shape : {r.shape}')
                all_recon.append(r)

    # Concatenate all scores and reconstructions
    all_scores = np.concatenate(all_scores, axis=0)  # (nb_seq, seq_len)
    all_recon = np.concatenate(all_recon, axis=0)

    print(f"Final scores shape: {all_scores.shape}, Final reconstructions shape: {all_recon.shape}")
    
    df_result = df.copy()
    df_result = df_result[['station', 'Date', target]]
    df_result["reconstruction"] = scaler.inverse_transform(all_recon)  # Inverse transform

    df_result["precipitation_scaled"] = data
    df_result["reconstruction_scaled"] = all_recon
    df_result["reconstruction_error"] = all_scores

    if threshold is not None:
        df_result["is_anomaly"] = df_result["reconstruction_error"] > threshold

    return df_result


In [17]:
import matplotlib.pyplot as plt
import json

def plot_loss_curves(setting, save_path="loss_curve.png"):
    path = "./test_results/" + setting + "/loss_history.json"
    with open(path, "r") as f:
        history = json.load(f)

    plt.figure(figsize=(8,5))
    plt.plot(history["train"], label="Train Loss", color="blue")
    plt.plot(history["val"], label="Validation Loss", color="orange")
    plt.plot(history["test"], label="Test Loss", color="green")
    
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss Curves")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Courbe sauvegardée : {save_path}")


In [6]:
target = "Cumul_precipitation"

In [51]:
args, model, threshold = load('anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el2_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0')
threshold

Use GPU: cuda:0


0.4681030124425892

In [7]:
df_raw = pd.read_excel('./dataset/ANAM/Stations_auto_uni.xlsx')

In [60]:
df_raw[target].value_counts()

0.0       230364
0.2         7209
0.4         4071
0.6         2931
0.8         2451
           ...  
3359.6         1
1561.8         1
1523.6         1
3098.6         1
16.9           1
Name: Cumul_precipitation, Length: 1808, dtype: int64

In [9]:
df_raw[target] = df_raw[target].str.replace(',', '.').astype(float)

## anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el2_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0

In [56]:
result = predict(args, df_raw, 'Cumul_precipitation', model, threshold)

Processing batch 1/3987 with shape (100, 1)
Processing batch 2/3987 with shape (100, 1)
Processing batch 3/3987 with shape (100, 1)
Processing batch 4/3987 with shape (100, 1)
Processing batch 5/3987 with shape (100, 1)
Processing batch 6/3987 with shape (100, 1)
Processing batch 7/3987 with shape (100, 1)
Processing batch 8/3987 with shape (100, 1)
Processing batch 9/3987 with shape (100, 1)
Processing batch 10/3987 with shape (100, 1)
Processing batch 11/3987 with shape (100, 1)
Processing batch 12/3987 with shape (100, 1)
Processing batch 13/3987 with shape (100, 1)
Processing batch 14/3987 with shape (100, 1)
Processing batch 15/3987 with shape (100, 1)
Processing batch 16/3987 with shape (100, 1)
Processing batch 17/3987 with shape (100, 1)
Processing batch 18/3987 with shape (100, 1)
Processing batch 19/3987 with shape (100, 1)
Processing batch 20/3987 with shape (100, 1)
Processing batch 21/3987 with shape (100, 1)
Processing batch 22/3987 with shape (100, 1)
Processing batch 23

In [47]:
result.head(10)

,station,Date,Cumul_precipitation,reconstruction,precipitation_scaled,reconstruction_scaled,reconstruction_error,is_anomaly
0,Arbinda,2019-01-01,0.0,-0.030980,-0.071822,-0.072266,1.977937e-07,False
1,Arbinda,2019-01-02,0.0,-0.012601,-0.071822,-0.072003,3.272215e-08,False
2,Arbinda,2019-01-03,0.0,-0.004919,-0.071822,-0.071892,4.985652e-09,False
3,Arbinda,2019-01-04,0.0,-0.020908,-0.071822,-0.072122,9.009248e-08,False
4,Arbinda,2019-01-05,0.0,-0.017947,-0.071822,-0.072079,6.637892e-08,False
5,Arbinda,2019-01-06,0.0,-0.009245,-0.071822,-0.071954,1.761385e-08,False
6,Arbinda,2019-01-07,0.0,-0.000486,-0.071822,-0.071829,4.873707e-11,False
7,Arbinda,2019-01-08,0.0,-0.010009,-0.071822,-0.071965,2.064522e-08,False
8,Arbinda,2019-01-09,0.0,-0.013177,-0.071822,-0.072011,3.578538e-08,False
9,Arbinda,2019-01-10,0.0,-0.007484,-0.071822,-0.071929,1.154279e-08,False


In [54]:
result["is_anomaly"].value_counts()

False    398105
True        559
Name: is_anomaly, dtype: int64

In [53]:
result.to_excel('2nd_stations_auto.xlsx', index=False)

In [9]:
result.to_excel("result.xlsx", index=False)

## anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el3_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0

In [61]:
args, model, threshold = load("anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el3_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0")

Use GPU: cuda:0


In [68]:
res = predict(args, df_raw, target, model, threshold)

Processing batch 1/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 2/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 3/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 4/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 5/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 6/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 7/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 8/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 9/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 10/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 11/3987 with shape (10

In [64]:
res.to_excel("3rd_stations_auto.xlsx", index=False)

In [65]:
res["is_anomaly"].value_counts()

False    398204
True        460
Name: is_anomaly, dtype: int64

## anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el3_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0

In [69]:
args, model, threshold = load("anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el3_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0")

Use GPU: cuda:0


In [71]:
res = predict(args, df_raw, target, model, threshold)

Processing batch 1/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 2/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 3/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 4/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 5/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 6/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 7/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 8/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 9/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 10/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 11/3987 with shape (10

In [72]:
res["is_anomaly"].value_counts()

False    398180
True        484
Name: is_anomaly, dtype: int64

In [77]:
res.to_excel("4th_station_auto_Topk4_elayer3_anomaly1.xlsx", index=True)

## anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm128_nh8_el3_dl1_df128_expand2_dc4_fc1_ebtimeF_dtTrue_test_0

In [14]:
setting = "anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm128_nh8_el3_dl1_df128_expand2_dc4_fc1_ebtimeF_dtTrue_test_0"

In [4]:
args, model, threshold = load("anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el3_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0")

Use GPU: cuda:0


In [10]:
res = predict(args, df_raw, target, model, threshold)

Processing batch 1/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 2/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 3/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 4/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 5/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 6/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 7/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 8/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 9/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 10/3987 with shape (100, 1)
loss shape (100, 1)
reconstruction shape : (100, 1)
Processing batch 11/3987 with shape (10

In [11]:
res["is_anomaly"].value_counts()

False    398173
True        491
Name: is_anomaly, dtype: int64

In [12]:
res.to_excel("5th_station_auto_dff128_dmodel128_Topk4_elayer3_anomaly10.xlsx", index=True)

In [18]:
plot_loss_curves(setting)

Courbe sauvegardée : loss_curve.png
